

通常，我们使用LVQ方法用在分类问题上。此部分最好见西瓜书的P204，有更详细的讲解。
此算法，最终是为了返回几个向量（原型向量），这n个向量将空间分为n个区域，令每个区域内样本的值距离当前区域的原型向量的距离不大于到其他原型向量的距离。
这种区域的划分称为Voronoi剖分。

codebook vector(是一系列数字，与你训练数据里的input与output相关的attribute一样)

example：
1.class 0,1
2.width
3.height
4.length

codebook vector(neuron):
1.class 0,1
2.width
3.height
4.length


LVQ跟KNN

通过在codebook vector里面进行寻找，通过Euclidean距离进行判断，找到BMU（Best Matching Unit）


1.选择一部分codebook vector。
2.竞争机制（codebook vector与训练实例（training pattern）一致的情况下。codebook vector向训练实例靠近，反之，则离远）
3.通过learning_rate控制移动的大小


x = x + learning_rate * (t - x)
其中：t是trainning input value， x是input variabile x。

4.对每个实例进行学习。

learning_rate = alpha(最初的学习率) * （1- (epoch/max_epoch)）
随着epoch增加，learning_rate逐步减小，学习地越来越精细。

计算两个向量之间的欧氏距离：

In [59]:
from math import sqrt

def calculate_euclidean_distance(row1,row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)


dataset = [[1.80,1.91,0],
           [1.85,2.11,0],
           [2.31,2.88,0],
           [3.54,-3.21,0],
           [3.66,3.12,0],
           [5.52,2.13,1],
           [6.32,1.46,1],
           [7.35,2.34,1],
           [7.78,3.26,1],
           [8.43,-0.34,1]
           ]

# row0 =dataset[0]
# for row in dataset:
#     distance = calculate_euclidean_distance(row0,row)
#     print(distance)

Best Matching Unit

1.计算距离（codebook vector 与 新的输入信息）
2.调用calculate_euclidean_distance
3.排序（考虑到数据类型）
4.选取BMU


In [60]:
def calculate_BMU(codebooks, test_row):
    distances = list()
    for codebook in codebooks:
        dist = calculate_euclidean_distance(codebook,test_row)
        distances.append((codebook,dist))
    distances.sort(key=lambda every_tuple : every_tuple[1]) # 以dist进行排序
    return distances[0][0]  # 第一个0是顺位上第一个，第二个0是将元组中的codebook部分拿出来。



In [61]:
test_row  = dataset[0]
bmu = calculate_BMU(dataset,test_row)
print(bmu)



[1.8, 1.91, 0]


In [62]:
# 训练我们的codebook vector

# 1.初始化（random feature）
# 2.在每一个epoch,通过 training pattern 进行对codebook vector更新（学习）
# 3.在每一个training pattern里面，每一个pattern feature如果与我们codebook vector一致的情况下，进行更新（更近，或者更远）
from random import randrange

def make_random_codebook(train):
    n_index = len(train)
    # print(n_index)
    n_features = len(train[0])
    # print(n_features)
    #train[][]中,每一次循环得出i值，按左右顺序执行一遍[][]。本例中，i=0时，[第1次随机][1]；i=1时，[第2次随机][2]；i=2时，[第3次随机][3]。
    #本例，相当于先在第1个属性里，从10个样本随机取一个值；然后在第2个属性里，从10个样本随机取一个值；最后在第3个属性里，从10个样本随机取一个值，赋值给codebook.
    codebook = [train[randrange(n_index)][i] for i in range(n_features)]
    # print(codebook)
    return codebook

# codebook = make_random_codebook(dataset)
# print(dataset)

In [63]:
def train_codebooks(train,n_codebooks,learn_rate,epochs):
    codebooks = [make_random_codebook(train) for i in range(n_codebooks)]
    print(codebooks)
    for epoch in range(epochs):
        rate = learn_rate * (1-(epoch/float(epochs)))

        sum_error = 0.0
        for row in train: #在train中遍历每个样本，更新bmu。外层还需要迭代epochs次数，一直在更新bmu中各属性。
            bmu = calculate_BMU(codebooks,row) #codebooks中与当前循环到的row中距离最近的为bmu
            print(bmu)
            for i in range(len(row)-1): #遍历bmu中的每个属性，下面对比row和bmu该属性的差异，并通过rate调整bmu的该属性
                error = row[i] - bmu[i] #当bmu是负值时，原if判断语句会出现不收敛的问题，sum_error可能会越来越大。
                sum_error += error**2
                if bmu[-1] >= row[-1]:
                    bmu[i] += rate * abs(error)
                else:
                    bmu[i] -= rate * abs(error)

                # 原判断语句
                # if bmu[-1] >= row[-1]:
                #     bmu[i] += rate * error
                # else:
                #     bmu[i] -= rate * error
        print('Our current epoch is【%d】, Our current learning rate is :【%.3f】, Our current sum of errors is 【%.3f】' % (epoch,rate,sum_error))
    return codebooks


In [64]:
learning_rate = 0.5
n_epoch = 1000
n_codebooks = 2
codebooks = train_codebooks(dataset,n_codebooks,learning_rate,n_epoch)
print('Our codebook is : %s' % codebooks)
#当多次运行时，sum of errors 可能会一直增加，原因很可能是因为样本数据过少和n_codebooks过小。


[[6.32, 1.46, 0], [1.8, 2.34, 1]]
[1.8, 2.34, 1]
[1.8, 2.5549999999999997, 1]
[1.8250000000000002, 2.7775, 1]
[6.32, 1.46, 0]
[2.0675, 2.82875, 1]
[7.710000000000001, 3.795, 0]
[6.615, 2.9625, 0]
[6.4675, 2.2112499999999997, 0]
[6.026250000000001, 2.1468749999999996, 0]
[5.149375000000001, 1.5903124999999996, 0]
Our current epoch is【0】, Our current learning rate is :【0.500】, Our current sum of errors is 【62.301】
[2.86375, 2.974375, 1]
[3.395093125, 3.5060303125000005, 1]
[4.1668671409375, 4.203347453593751, 1]
[3.5090625000000015, 0.6251562499999997, 0]
[3.5245157812500008, 2.5408167968749997, 0]
[3.5921901485156256, 2.8301188068359373, 0]
[5.094372277835781, 4.864359506663829, 1]
[5.706573325056809, 6.5648370802424125, 1]
[2.6292491276991807, 2.4804094628213864, 0]
[0.05644906698492136, 2.091003989500669, 0]
Our current epoch is【1】, Our current learning rate is :【0.499】, Our current sum of errors is 【167.876】
[-4.12613962405611, 0.876717496745085, 0]
[-1.1689959516521111, 1.3923254658